# 관측지점 데이터 추출

In [23]:
import requests
import pandas as pd
from io import StringIO
import os
from dotenv import load_dotenv
load_dotenv()

WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

# API URL
url = "https://apihub.kma.go.kr/api/typ01/url/stn_inf.php"
params = {
    "inf": "SFC",                # 지상 관측
    "stn": "",                   # 전체 지점
    "tm": "202211300900",        # 시간 (YYYYMMDDhhmm)
    "help": "1",                 # 컬럼명 포함
    "authKey": WEATHER_API_KEY   # 본인 인증키로 교체
}

# 요청 및 응답
response = requests.get(url, params=params)

# 응답 파싱
if response.status_code == 200:
    csv_text = response.text
    df = pd.read_csv(StringIO(csv_text), comment='#', sep='\t')
    print(df.head())  # 결과 확인
else:
    print(f"에러 발생: {response.status_code}")

df.to_csv('data/기상청API_기상관측지점.csv', encoding='cp949')

     90  128.56473000   38.25085000 35100        17.53     18.73      1.70     10.00      1.40  90 속초                 Sokcho               11D20401 5182033035 ----
0     93  127.75443000   37.94738000 31201       ...                                                                                                               
1     95  127.30420000   38.14787000 31110       ...                                                                                                               
2     98  127.06070000   37.90188000 22200       ...                                                                                                               
3     99  126.76648000   37.88589000 22300       ...                                                                                                               
4    100  128.71834000   37.67713000 35400       ...                                                                                                               


# 관측지점별 기간 데이터 추출

In [36]:
import requests
import pandas as pd
import time

from io import StringIO
from dotenv import load_dotenv
load_dotenv()

WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")
tm1 = '20180101'
tm2 = '20250531'

df_stn = pd.read_csv('data/기상청API_기상관측지점.csv', encoding='cp949')
stn_list = df_stn.iloc[:,0].values

# API URL 및 파라미터 설정
url = "https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php"
data_list = []

for stn in stn_list:
    params = {
        "tm1": tm1,
        "tm2": tm2,
        "stn": stn,  # 서울
        "help": "1",
        "authKey": WEATHER_API_KEY # 실제 인증키로 교체
    }

    # 요청
    res = requests.get(url, params=params)
    time.sleep(0.2)
    text = res.text

    # 줄 단위로 나누기
    lines = text.strip().splitlines()

    # 주석 줄 제거 및 데이터 줄만 추출
    data_lines = [line for line in lines if not line.startswith("#")]

    # 각 줄을 공백으로 분리
    data = [line.split() for line in data_lines]
    data_list.extend(data)

# DataFrame 생성
df = pd.DataFrame(data_list)

# 컬럼명 지정 (일부 주요 컬럼만 예시로 명시)
column_names = [
    'TM', 'STN', 'WS_AVG', 'WR_DAY', 'WD_MAX', 'WS_MAX', 'WS_MAX_TM', 'WD_INS', 'WS_INS', 'WS_INS_TM', 
    'TA_AVG', 'TA_MAX', 'TA_MAX_TM', 'TA_MIN', 'TA_MIN_TM', 'TD_AVG', 'TS_AVG', 'TG_MIN', 
    'HM_AVG', 'HM_MIN', 'HM_MIN_TM', 'PV_AVG', 'EV_S', 'EV_L', 'FG_DUR', 'PA_AVG', 'PS_AVG', 'PS_MAX', 
    'PS_MAX_TM', 'PS_MIN', 'PS_MIN_TM', 'CA_TOT', 'SS_DAY', 'SS_DUR', 'SS_CMB', 'SI_DAY', 'SI_60M_MAX', 
    'SI_60M_MAX_TM', 'RN_DAY', 'RN_D99', 'RN_DUR', 'RN_60M_MAX', 'RN_60M_MAX_TM', 'RN_10M_MAX', 'RN_10M_MAX_TM', 
    'RN_POW_MAX', 'RN_POW_MAX_TM', 'SD_NEW', 'SD_NEW_TM', 'SD_MAX', 'SD_MAX_TM', 'TE_05', 'TE_10', 'TE_15', 'TE_30', 'TE_50'
]

# 컬럼 개수 맞춰서 지정 (컬럼 수 < 데이터 수 일 경우 잘라내기)
df = df.iloc[:, :len(column_names)]
df.columns = column_names

# 저장
df.to_csv(f"기상청_전국_일기요소_{tm1}-{tm2}.csv", index=False, encoding='cp949')

df.head()


,TM,STN,WS_AVG,WR_DAY,WD_MAX,WS_MAX,WS_MAX_TM,WD_INS,WS_INS,WS_INS_TM,...,RN_POW_MAX_TM,SD_NEW,SD_NEW_TM,SD_MAX,SD_MAX_TM,TE_05,TE_10,TE_15,TE_30,TE_50
0,20180101,90,2.6,2256,29,6.4,1250,32,11.1,1331,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
1,20180102,90,2.9,2482,29,7.0,1342,29,11.9,1146,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
2,20180103,90,1.6,1403,29,3.7,2331,29,5.9,2331,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
3,20180104,90,1.5,1331,36,3.3,1335,29,5.5,515,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
4,20180105,90,1.2,1027,2,2.8,1407,29,5.6,1802,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0


,TM,STN,WS_AVG,WR_DAY,WD_MAX,WS_MAX,WS_MAX_TM,WD_INS,WS_INS,WS_INS_TM,...,RN_POW_MAX_TM,SD_NEW,SD_NEW_TM,SD_MAX,SD_MAX_TM,TE_05,TE_10,TE_15,TE_30,TE_50
0,20180101,108,1.4,1188,29,3.8,1634,32,7.0,1633,...,-9,-9.0,-9,-9.0,-9,2.7,5.9,9.0,15.5,17.6
1,20180102,108,1.8,1573,29,4.9,1355,27,8.5,1256,...,-9,-9.0,-9,-9.0,-9,2.7,5.8,8.9,15.4,17.6
2,20180103,108,2.2,1943,27,3.5,1108,32,6.6,1340,...,-9,-9.0,-9,-9.0,-9,2.6,5.7,8.9,15.3,17.5
3,20180104,108,1.4,1168,32,3.5,550,32,5.8,547,...,-9,-9.0,-9,-9.0,-9,2.6,5.6,8.7,15.2,17.5
4,20180105,108,1.7,1450,27,3.6,1712,25,8.0,1443,...,-9,-9.0,-9,-9.0,-9,2.6,5.5,8.7,15.1,17.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,20250527,108,1.9,1673,29,4.5,1817,32,7.1,1815,...,-9,-9.0,-9,-9.0,-9,19.6,17.6,16.2,13.8,13.4
2704,20250528,108,1.8,1595,27,4.7,1632,27,7.6,1646,...,-9,-9.0,-9,-9.0,-9,19.9,17.6,16.3,13.9,13.4
2705,20250529,108,2.2,1939,25,4.5,1802,25,7.8,1738,...,-9,-9.0,-9,-9.0,-9,20.1,17.8,16.4,14.0,13.4
2706,20250530,108,1.7,1470,25,4.2,1848,27,6.5,1840,...,-9,-9.0,-9,-9.0,-9,20.4,18.0,16.5,14.0,13.4
